In [1]:
import numpy as np
import matplotlib.pyplot as plt
import camb
from camb import model, initialpower

In this notebook, we will train our iterative emulator on a simple scenario of linear Power spectrum, i.e, we assume that our data vector is the _linear_ power spectrum, $P(k)$. 

We will use our neural network emulator to predict the power spectrum as a function of cosmology. 

We use `camb` to predict the linear power spectrum, as defined below. 

In [2]:
def get_Pk(cosmo_pars):
    As, ns, H0, ombh2, omch2 = cosmo_pars
    #Now get matter power spectra and sigma8 at redshift 0 and 0.8
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2)
    pars.InitPower.set_params(As=As, ns=ns)
    #Note non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)

    #Linear spectra
    pars.NonLinear = model.NonLinear_none
    results = camb.get_results(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-3, maxkh=1, npoints = 2000)
    s8 = np.array(results.get_sigma8())
    
    return kh, pk[0], s8

To use the emulator, we need a function to compute the data vector as a function of cosmological parameters. 

For your specific case, you can simply replace this function along with the log probability function to use the emulator as proposed here.

In [3]:
def compute_datavector(cosmo_pars):
    kh, pk, _ = get_Pk(cosmo_pars)
    # We use the logarithm of the power spectrum as it is easier to train
    return np.log(pk)

In [4]:
# We use the following fiducial parameters in our analysis

cosmo_pars_fid = np.array([2e-9, 0.97, 70., 0.0228528, 0.1199772])

kh_fid, pk_fid, _ = get_Pk(cosmo_pars_fid)

## Initialize a Latin Hypercube

We begin our emulator training, we need a Latin Hypercube sample with the prior range.

In [5]:
from pyDOE import lhs

N_dim = 5

cosmo_prior = np.array([[1.2e-9, 2.7e-9], # As
                       [0.87, 1.07],      # ns
                       [60, 85],          # H0
                       [0.01, 0.04],      # omega_b
                       [0.01, 0.3]])      # omega_c

def get_cosmo_lhs_samples(N_samples, cosmo_prior):
    lhs_samples = lhs(N_dim, N_samples)
    cosmo_samples = cosmo_prior[:,0] + (cosmo_prior[:,1] - cosmo_prior[:,0]) * lhs_samples
    return cosmo_samples

In [6]:
from tqdm import tqdm
from multiprocessing import Pool

def calculate_datavector_batch(cosmo_samples):
    """
    Function to calculate the data vectors for a batch of training samples
    """
    train_dv_list = []
    with Pool() as p:
        train_dv_list = list(tqdm(p.imap(compute_datavector, cosmo_samples), total=len(cosmo_samples)))
    return np.array(train_dv_list)    

In [7]:
N_train_samples = 1000

train_cosmo_samples = get_cosmo_lhs_samples(N_train_samples, cosmo_prior)

train_dv_arr = calculate_datavector_batch(train_cosmo_samples)

100%|██████████| 1000/1000 [06:31<00:00,  2.55it/s]


In [9]:
N_test_samples = 400

test_cosmo_samples = get_cosmo_lhs_samples(N_test_samples, cosmo_prior)
test_dv_arr = calculate_datavector_batch(test_cosmo_samples)

100%|██████████| 400/400 [02:39<00:00,  2.52it/s]


### PCA

Take the PCAs of the training samples and then train your emulator to predict the PCAs

In [10]:
from sklearn.decomposition import PCA

In [11]:
def get_Pk(pca_coeffs, pca):
    """
    Function to get the power spectrum from the PCA coefficients
    :pca_coefficient: PCA coefficients
    :pca: sklearn PCA object
    """
    log_Pk_pred  = pca.inverse_transform(pca_coeffs)
    return np.exp(log_Pk_pred)

In [12]:
n_pca = 5

pca = PCA(n_pca)
pca.fit(train_dv_arr)

train_pca_coeff = pca.transform(train_dv_arr)

train_pca_mean = train_pca_coeff.mean(0)
train_pca_std  = train_pca_coeff.std(0)

In [13]:
from emulator import NNEmulator
import torch

In [ ]:
emu = NNEmulator(N_dim, n_pca, train_pca_mean, train_pca_std)
emu.train(torch.Tensor(train_cosmo_samples), torch.Tensor(train_pca_coeff))

Loss: 0.08990950137376785:  42%|████▏     | 42/100 [00:27<00:29,  1.95it/s] 

In [ ]:
test_pca_pred = emu.predict(torch.Tensor(test_cosmo_samples))

In [ ]:
test_pk_pred = get_Pk(test_pca_pred, pca)
test_pk_arr  = np.exp(test_dv_arr)

In [ ]:
DeltaP_frac = np.abs((test_pk_pred - test_pk_arr) / test_pk_arr)

In [ ]:
plt.ylim(0., 0.15)
plt.semilogx(kh_fid, np.median(DeltaP_frac, axis=0))
plt.show()

With the emulator to predict 5 PCA components, we get median accuracy of $\sim \mathcal{O}(5\%)$. Increasing the number of training samples will likely improve the accuracy.